<a href="https://colab.research.google.com/github/oltsaaa/Python/blob/main/Portfolio1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SQL запит: https://docs.google.com/document/d/1NFmir-y6ExjVbgnFg4jfYAyQmAswv3QDTGxeIMjnunQ/edit?usp=sharing


CSV result of SQL query: https://drive.google.com/file/d/1JCPN8-Q7olnOdfsKQWfVrrLRkuJ_Ynn-/view?usp=sharing


Створи блокнот у Google Collab та порахуй статистичну значущість для чотирьох метрик:
add_payment_info / session
add_shipping_info / session
begin_checkout / session
new_accounts / session
Розраховуй значущість в тоталі по тест

In [ ]:

from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [ ]:

%cd /content/drive/MyDrive/MateColab


/content/drive/MyDrive/MateColab


In [ ]:
data = pd.read_csv("portfolio1.csv")
data.head()


,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-05,Cambodia,desktop,Asia,Undefined,2,1,new_acc,1
1,2020-11-05,Cambodia,desktop,Asia,Undefined,1,1,new_acc,1
2,2021-01-19,Cambodia,tablet,Asia,Organic Search,4,2,new_acc,1
3,2021-01-09,Kazakhstan,mobile,Asia,Paid Search,4,1,new_acc,1
4,2020-12-15,Qatar,desktop,Asia,Direct,4,1,new_acc,1


In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm


# Define metrics mapping
metrics = {
    "add_payment_info": ("add_payment_info", "session"),
    "add_shipping_info": ("add_shipping_info", "session"),
    "begin_checkout": ("begin_checkout", "session"),
    "new_acc": ("new_acc", "session")
}

# List for storing results
test_results = []

# Calculate z-test for each test in the dataset
for test_number in data['test'].unique():
    test_data = data[data['test'] == test_number]
    for metric_name, (numerator_event, denominator_event) in metrics.items():
        # Sum up counts for the numerator and denominator in the test and control groups
        numerator_count = test_data[(test_data['event_name'] == numerator_event) & (test_data['test_group'] == 2)]['value'].sum()
        denominator_count = test_data[(test_data['event_name'] == denominator_event) & (test_data['test_group'] == 2)]['value'].sum()
        numerator_count_control = test_data[(test_data['event_name'] == numerator_event) & (test_data['test_group'] == 1)]['value'].sum()
        denominator_count_control = test_data[(test_data['event_name'] == denominator_event) & (test_data['test_group'] == 1)]['value'].sum()
        # Check for valid denominator values
        if denominator_count > 0 and denominator_count_control > 0:
            # Conversion rates for test and control groups
            conversion_rate_test = numerator_count / denominator_count
            conversion_rate_control = numerator_count_control / denominator_count_control
            metric_change = (conversion_rate_test - conversion_rate_control) / conversion_rate_control * 100  # Percentage change
            # Perform z-test for statistical significance
            z_stat, p_value = sm.stats.proportions_ztest(
                [numerator_count, numerator_count_control],
                [denominator_count, denominator_count_control]
            )
            # Append results to the list
            test_results.append({
                "test_number": test_number,
                "metric": metric_name,
                "numerator_event": numerator_event,
                "denominator_event": denominator_event,
                "numerator_count_test": numerator_count,
                "denominator_count_test": denominator_count,
                "conversion_rate_test": conversion_rate_test,
                "numerator_count_control": numerator_count_control,
                "denominator_count_control": denominator_count_control,
                "conversion_rate_control": conversion_rate_control,
                "metric_change_%": metric_change,
                "z_stat": z_stat,
                "p_value": p_value,
                "significant": p_value < 0.05
            })


# Convert results to DataFrame
test_results = pd.DataFrame(test_results)

# Display the results
print(test_results)



    test_number             metric    numerator_event denominator_event  \
0             2   add_payment_info   add_payment_info           session   
1             2  add_shipping_info  add_shipping_info           session   
2             2     begin_checkout     begin_checkout           session   
3             2            new_acc            new_acc           session   
4             1   add_payment_info   add_payment_info           session   
5             1  add_shipping_info  add_shipping_info           session   
6             1     begin_checkout     begin_checkout           session   
7             1            new_acc            new_acc           session   
8             4   add_payment_info   add_payment_info           session   
9             4  add_shipping_info  add_shipping_info           session   
10            4     begin_checkout     begin_checkout           session   
11            4            new_acc            new_acc           session   
12            3   add_pay

In [ ]:
test_results.to_csv("test_results.csv", index=False, encoding="utf-8")
from google.colab import files
files.download("test_results.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>